In [ ]:
import os
import cv2
import tqdm
import json
import pickle
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from utils.dataset_utils import *
from collections import defaultdict
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
# Define paths
data_folder = rf'..\data\1. Data Processing\raw\pred_attack'
tensor_folder = rf'..\data\1. Data Processing\processed\video\expert_tensors\windows'
window_len = 10

pred_tensors_list = []
prey_tensors_list = []

# process each json file in the data folder
for file in os.listdir(data_folder):
    if file.endswith(".json"):
        file_path = os.path.join(data_folder, file)

        # load raw points data
        with open(file_path, 'r') as f:
            data = json.load(f)
        
        # scale data points and separate predators and preys
        pairs = [scale_data(pts) for pts in data]
        pred_pts = [p for p, _ in pairs] 
        prey_pts = [q for _, q in pairs]

        # assign identities using Hungarian algorithm
        pred_ordered = hungarian_assign(pred_pts)
        prey_ordered = hungarian_assign(prey_pts)

        # compute velocities for agents from assigned positions
        pred_velocities = get_velocity(pred_ordered)
        prey_velocities = get_velocity(prey_ordered)

        # get records for expert tensors (same format as video pipeline)
        records = get_records(pred_ordered, prey_ordered, pred_velocities, prey_velocities)

        # convert records into expert tensors
        pred_tensor, prey_tensor = get_hl_expert_tensors(records, max_speed=10)

        # extract windows from tensors
        pred_windows = extract_windows(pred_tensor, window_len=window_len)
        prey_windows = extract_windows(prey_tensor, window_len=window_len)
        pred_tensors_list.append(pred_windows)
        prey_tensors_list.append(prey_windows)

# concatenate all windows from all files
pred_tensors = torch.cat(pred_tensors_list, dim=0)
prey_tensors = torch.cat(prey_tensors_list, dim=0)

# add flag feature to prey tensor
n, window, agents, neighs, feature = prey_tensors.shape
flag = torch.zeros((n, window, agents, neighs, 1), dtype=prey_tensors.dtype, device=prey_tensors.device)
flag[:, :1, 0] = 1
prey_tensors = torch.cat([flag, prey_tensors], dim=-1)

# save tensors
pred_path = os.path.join(tensor_folder, f"{window_len} windows" , f"pred_tensors_hl_w{window_len}_n{len(pred_tensors)}.pkl")
prey_path = os.path.join(tensor_folder, f"{window_len} windows" , f"prey_tensors_hl_w{window_len}_n{len(prey_tensors)}.pkl")

torch.save(pred_tensors, pred_path)
torch.save(prey_tensors, prey_path)

print("Pred Tensors Shape:", tuple(pred_tensors.shape))
print("Prey Tensors Shape:", tuple(prey_tensors.shape))

Pred Tensors Shape: (769, 10, 1, 32, 5)
Prey Tensors Shape: (769, 10, 32, 32, 6)
